In [48]:
import pandas as pd
import os
import time
import numpy as np
import scipy.optimize as sco
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import LabelEncoder

In [49]:
def readFile():
    info = pd.read_csv('/home/team36/fanjiakuan/data/adjusted_net_value.csv')
    return info
info = readFile()
info.rename(columns = {'Unnamed: 0': 'date'}, inplace = True)
info['date'] = pd.to_datetime(info['date'])
info = info.set_index('date')
info_select = info['2016-06-14':'2018-11-30']
info_select

In [7]:

'将序列转换成监督学习问题'
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


In [ ]:
start_time = time.clock()
print('-----------starting-----------')
# 设计网络
model = Sequential()
model.add(LSTM(50, input_shape=(1, 5)))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

# com_df_len = len(com_df)

y_predict = []
count = 1
for i in info_select.columns:
    
#     encoder = LabelEncoder()              # 整数编码方向 

#     values['subscribe_status'] = encoder.fit_transform(values['subscribe_status'])
#     values['redeem_status'] = encoder.fit_transform(values['redeem_status'])
    values = info_select[i].astype('float32')         # 确保所有列数据是浮动的
#     values = values.fillna(1)
    values = np.array(values).reshape(-1,1)

    scaler = MinMaxScaler(feature_range=(0, 1))        # 归一化特征
    scaled = scaler.fit_transform(values)

    reframed = series_to_supervised(scaled, 5, 1)      #转换为监督学习问题
    # print(reframed.head())
   
    values = reframed.values
#     split_pos = int(len(values) * 0.9)
    train = values[:-22, :]
    test = values[-22:, :]
    
    train_X, train_y = train[:, :-1], train[:, -1]
    test_X, test_y = test[:, :-1], test[:, -1]
    # 把输入重塑成3D格式 [样例， 时间步, 特征]
    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    # print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)


    
    # 拟合网络
    history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=1, shuffle=False)
    # 绘制历史数据
#     pyplot.plot(history.history['loss'], label='train')
#     pyplot.plot(history.history['val_loss'], label='test')
#     pyplot.legend()
#     pyplot.show()

    # 预测
    yhat = model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
    # 反向缩放预测值
    inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
#     print('inv_yhat原始:',inv_yhat)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,0]
    # 反向缩放实际值
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]
    
    # 计算RMSE
    rmse = sqrt(mean_squared_error(inv_y, inv_yhat))

#     for i in range(len(inv_yhat)):
    y_predict.append(inv_yhat)

    if count%30==0:
        y_predict_df = pd.DataFrame(y_predict)
        y_predict_df.to_csv('../output/output_mouth.csv', mode = 'a', header = False)
        del y_predict,y_predict_df
        y_predict = []
    
   
    print('第%s个数，还剩%s个没有训练' % (count,len(info_select.columns)-count))

    print('Test RMSE: %.3f' % rmse)
    count = count + 1
    
y_predict_df = pd.DataFrame(np.array(y_predict))
y_predict_df.to_csv('../output/output_mouth.csv', mode = 'a', header = False)


end_time = time.clock()
used_time = start_time - end_time
print('----------finished------------')
print('used time:%s s'% used_time)


-----------starting-----------
Train on 582 samples, validate on 22 samples
Epoch 1/50
582/582 [==============================] - 2s 3ms/sample - loss: 0.4257 - val_loss: 0.7243
Epoch 2/50
582/582 [==============================] - 0s 96us/sample - loss: 0.3392 - val_loss: 0.5844
Epoch 3/50
582/582 [==============================] - 0s 87us/sample - loss: 0.2481 - val_loss: 0.4309
Epoch 4/50
582/582 [==============================] - 0s 82us/sample - loss: 0.1488 - val_loss: 0.2588
Epoch 5/50
582/582 [==============================] - 0s 80us/sample - loss: 0.0560 - val_loss: 0.0866
Epoch 6/50
582/582 [==============================] - 0s 85us/sample - loss: 0.0516 - val_loss: 0.0284
Epoch 7/50
582/582 [==============================] - 0s 85us/sample - loss: 0.0604 - val_loss: 0.0445
Epoch 8/50
582/582 [==============================] - 0s 83us/sample - loss: 0.0512 - val_loss: 0.0665
Epoch 9/50
582/582 [==============================] - 0s 84us/sample - loss: 0.0430 - val_loss: 0.055

582/582 [==============================] - 0s 83us/sample - loss: 0.0208 - val_loss: 0.0185
Epoch 29/50
582/582 [==============================] - 0s 87us/sample - loss: 0.0245 - val_loss: 0.0193
Epoch 30/50
582/582 [==============================] - 0s 91us/sample - loss: 0.0212 - val_loss: 0.0185
Epoch 31/50
582/582 [==============================] - 0s 85us/sample - loss: 0.0243 - val_loss: 0.0193
Epoch 32/50
582/582 [==============================] - 0s 85us/sample - loss: 0.0210 - val_loss: 0.0185
Epoch 33/50
582/582 [==============================] - 0s 86us/sample - loss: 0.0236 - val_loss: 0.0194
Epoch 34/50
582/582 [==============================] - 0s 85us/sample - loss: 0.0208 - val_loss: 0.0186
Epoch 35/50
582/582 [==============================] - 0s 86us/sample - loss: 0.0231 - val_loss: 0.0194
Epoch 36/50
582/582 [==============================] - 0s 87us/sample - loss: 0.0203 - val_loss: 0.0186
Epoch 37/50
582/582 [==============================] - 0s 83us/sample - loss

582/582 [==============================] - 0s 83us/sample - loss: 0.0260 - val_loss: 0.0291
Epoch 6/50
582/582 [==============================] - 0s 82us/sample - loss: 0.0259 - val_loss: 0.0291
Epoch 7/50
582/582 [==============================] - 0s 83us/sample - loss: 0.0257 - val_loss: 0.0291
Epoch 8/50
582/582 [==============================] - 0s 85us/sample - loss: 0.0256 - val_loss: 0.0289
Epoch 9/50
582/582 [==============================] - 0s 79us/sample - loss: 0.0254 - val_loss: 0.0287
Epoch 10/50
582/582 [==============================] - 0s 81us/sample - loss: 0.0255 - val_loss: 0.0286
Epoch 11/50
582/582 [==============================] - 0s 82us/sample - loss: 0.0252 - val_loss: 0.0286
Epoch 12/50
582/582 [==============================] - 0s 83us/sample - loss: 0.0250 - val_loss: 0.0284
Epoch 13/50
582/582 [==============================] - 0s 83us/sample - loss: 0.0254 - val_loss: 0.0283
Epoch 14/50
582/582 [==============================] - 0s 83us/sample - loss: 0.

582/582 [==============================] - 0s 79us/sample - loss: 0.0240 - val_loss: 0.0203
Epoch 34/50
582/582 [==============================] - 0s 89us/sample - loss: 0.0243 - val_loss: 0.0204
Epoch 35/50
582/582 [==============================] - 0s 86us/sample - loss: 0.0241 - val_loss: 0.0204
Epoch 36/50
582/582 [==============================] - 0s 81us/sample - loss: 0.0239 - val_loss: 0.0204
Epoch 37/50
582/582 [==============================] - 0s 89us/sample - loss: 0.0238 - val_loss: 0.0203
Epoch 38/50
582/582 [==============================] - 0s 84us/sample - loss: 0.0237 - val_loss: 0.0202
Epoch 39/50
582/582 [==============================] - 0s 87us/sample - loss: 0.0241 - val_loss: 0.0203
Epoch 40/50
582/582 [==============================] - 0s 81us/sample - loss: 0.0246 - val_loss: 0.0203
Epoch 41/50
582/582 [==============================] - 0s 84us/sample - loss: 0.0240 - val_loss: 0.0202
Epoch 42/50
582/582 [==============================] - 0s 82us/sample - loss

582/582 [==============================] - 0s 80us/sample - loss: 0.0238 - val_loss: 0.0197
Epoch 11/50
582/582 [==============================] - 0s 84us/sample - loss: 0.0238 - val_loss: 0.0197
Epoch 12/50
582/582 [==============================] - 0s 87us/sample - loss: 0.0238 - val_loss: 0.0197
Epoch 13/50
582/582 [==============================] - 0s 83us/sample - loss: 0.0238 - val_loss: 0.0197
Epoch 14/50
582/582 [==============================] - 0s 81us/sample - loss: 0.0237 - val_loss: 0.0197
Epoch 15/50
582/582 [==============================] - 0s 82us/sample - loss: 0.0237 - val_loss: 0.0197
Epoch 16/50
582/582 [==============================] - 0s 87us/sample - loss: 0.0237 - val_loss: 0.0196
Epoch 17/50
582/582 [==============================] - 0s 81us/sample - loss: 0.0237 - val_loss: 0.0197
Epoch 18/50
582/582 [==============================] - 0s 85us/sample - loss: 0.0236 - val_loss: 0.0196
Epoch 19/50
582/582 [==============================] - 0s 82us/sample - loss

...

582/582 [==============================] - 0s 86us/sample - loss: 0.0113 - val_loss: 0.0141
Epoch 27/50
582/582 [==============================] - 0s 87us/sample - loss: 0.0113 - val_loss: 0.0142
Epoch 28/50
582/582 [==============================] - 0s 84us/sample - loss: 0.0113 - val_loss: 0.0141
Epoch 29/50
582/582 [==============================] - 0s 82us/sample - loss: 0.0113 - val_loss: 0.0141
Epoch 30/50
582/582 [==============================] - 0s 86us/sample - loss: 0.0113 - val_loss: 0.0141
Epoch 31/50
582/582 [==============================] - 0s 91us/sample - loss: 0.0113 - val_loss: 0.0140
Epoch 32/50
582/582 [==============================] - 0s 91us/sample - loss: 0.0113 - val_loss: 0.0141
Epoch 33/50
582/582 [==============================] - 0s 93us/sample - loss: 0.0113 - val_loss: 0.0140
Epoch 34/50
582/582 [==============================] - 0s 87us/sample - loss: 0.0113 - val_loss: 0.0140
Epoch 35/50
582/582 [==============================] - 0s 87us/sample - loss

582/582 [==============================] - 0s 79us/sample - loss: 0.0165 - val_loss: 0.0156
Epoch 4/50
582/582 [==============================] - 0s 88us/sample - loss: 0.0162 - val_loss: 0.0159
Epoch 5/50
582/582 [==============================] - 0s 82us/sample - loss: 0.0159 - val_loss: 0.0156
Epoch 6/50
582/582 [==============================] - 0s 86us/sample - loss: 0.0157 - val_loss: 0.0156
Epoch 7/50
582/582 [==============================] - 0s 83us/sample - loss: 0.0159 - val_loss: 0.0155
Epoch 8/50
582/582 [==============================] - 0s 87us/sample - loss: 0.0157 - val_loss: 0.0153
Epoch 9/50
582/582 [==============================] - 0s 84us/sample - loss: 0.0156 - val_loss: 0.0155
Epoch 10/50
582/582 [==============================] - 0s 87us/sample - loss: 0.0158 - val_loss: 0.0153
Epoch 11/50
582/582 [==============================] - 0s 84us/sample - loss: 0.0157 - val_loss: 0.0155
Epoch 12/50
582/582 [==============================] - 0s 85us/sample - loss: 0.01

582/582 [==============================] - 0s 87us/sample - loss: 0.0168 - val_loss: 0.0179
Epoch 32/50
582/582 [==============================] - 0s 82us/sample - loss: 0.0168 - val_loss: 0.0178
Epoch 33/50
582/582 [==============================] - 0s 81us/sample - loss: 0.0169 - val_loss: 0.0181
Epoch 34/50
582/582 [==============================] - 0s 80us/sample - loss: 0.0170 - val_loss: 0.0179
Epoch 35/50
582/582 [==============================] - 0s 79us/sample - loss: 0.0169 - val_loss: 0.0178
Epoch 36/50
582/582 [==============================] - 0s 81us/sample - loss: 0.0172 - val_loss: 0.0182
Epoch 37/50
582/582 [==============================] - 0s 82us/sample - loss: 0.0170 - val_loss: 0.0181
Epoch 38/50
582/582 [==============================] - 0s 83us/sample - loss: 0.0169 - val_loss: 0.0179
Epoch 39/50
582/582 [==============================] - 0s 91us/sample - loss: 0.0169 - val_loss: 0.0180
Epoch 40/50
582/582 [==============================] - 0s 92us/sample - loss

582/582 [==============================] - 0s 84us/sample - loss: 0.0237 - val_loss: 0.0251
Epoch 9/50
582/582 [==============================] - 0s 81us/sample - loss: 0.0228 - val_loss: 0.0299
Epoch 10/50
582/582 [==============================] - 0s 83us/sample - loss: 0.0268 - val_loss: 0.0251
Epoch 11/50
582/582 [==============================] - 0s 83us/sample - loss: 0.0249 - val_loss: 0.0306
Epoch 12/50
582/582 [==============================] - 0s 83us/sample - loss: 0.0295 - val_loss: 0.0251
Epoch 13/50
582/582 [==============================] - 0s 87us/sample - loss: 0.0264 - val_loss: 0.0304
Epoch 14/50
582/582 [==============================] - 0s 83us/sample - loss: 0.0310 - val_loss: 0.0251
Epoch 15/50
582/582 [==============================] - 0s 87us/sample - loss: 0.0251 - val_loss: 0.0293
Epoch 16/50
582/582 [==============================] - 0s 91us/sample - loss: 0.0283 - val_loss: 0.0254
Epoch 17/50
582/582 [==============================] - 0s 88us/sample - loss:

582/582 [==============================] - 0s 87us/sample - loss: 0.0089 - val_loss: 0.0082
Epoch 37/50
582/582 [==============================] - 0s 84us/sample - loss: 0.0099 - val_loss: 0.0054
Epoch 38/50
582/582 [==============================] - 0s 86us/sample - loss: 0.0089 - val_loss: 0.0081
Epoch 39/50
582/582 [==============================] - 0s 85us/sample - loss: 0.0098 - val_loss: 0.0054
Epoch 40/50
582/582 [==============================] - 0s 84us/sample - loss: 0.0089 - val_loss: 0.0084
Epoch 41/50
582/582 [==============================] - 0s 89us/sample - loss: 0.0098 - val_loss: 0.0053
Epoch 42/50
582/582 [==============================] - 0s 79us/sample - loss: 0.0087 - val_loss: 0.0094
Epoch 43/50
582/582 [==============================] - 0s 88us/sample - loss: 0.0092 - val_loss: 0.0056
Epoch 44/50
582/582 [==============================] - 0s 87us/sample - loss: 0.0087 - val_loss: 0.0091
Epoch 45/50
582/582 [==============================] - 0s 91us/sample - loss

In [26]:
# predict_df = pd.read_csv('../output/output2.csv', header = None)
# predict_df.drop([0], axis=1, inplace = True)
predict = predict_df.T

In [45]:
index = [['2018-12-11', '2018-12-12', '2018-12-13', '2018-12-14', '2018-12-17', '2018-12-18', '2018-12-19', '2018-12-20']]
predict.set_index(index)
predict


,519661,000061,398061,519995,519095,000395,470059,000259,000059,519692,...,202025,070025,159925,519725,000025,519025,161625,270025,180025,110025
1,1.652255,0.561115,2.205736,2.392376,1.945991,1.214544,1.370850,1.872195,1.212983,4.096904,...,1.135406,1.480372,1.177575,1.302995,1.410857,1.099173,0.787824,1.465087,1.467835,0.840956
2,1.652716,0.559109,2.210475,2.395306,1.950106,1.214621,1.369794,1.883440,1.217350,4.124184,...,1.143848,1.480084,1.186037,1.302732,1.410873,1.108182,0.788260,1.479205,1.470785,0.847195
3,1.653606,0.557623,2.216013,2.397270,1.956941,1.215206,1.370983,1.886210,1.215719,4.131520,...,1.147213,1.480184,1.190233,1.302850,1.410779,1.107896,0.788316,1.477969,1.469316,0.849066
4,1.653704,0.558961,2.225332,2.415637,1.968671,1.214795,1.377941,1.904401,1.222779,4.192462,...,1.162498,1.480108,1.208818,1.302729,1.411870,1.125212,0.792939,1.488967,1.472001,0.855138
5,1.652903,0.557526,2.223805,2.409332,1.955560,1.214625,1.374068,1.880105,1.202408,4.149642,...,1.141697,1.478915,1.189105,1.301329,1.411702,1.104615,0.789472,1.467526,1.466618,0.843043
6,1.651095,0.555529,2.223164,2.397780,1.951212,1.213823,1.367357,1.861262,1.188466,4.087891,...,1.138229,1.479074,1.186934,1.301432,1.410619,1.099154,0.787608,1.455193,1.464143,0.843273
7,1.648900,0.553896,2.217948,2.382764,1.942166,1.213005,1.359174,1.841436,1.179948,4.039001,...,1.128285,1.477747,1.171357,1.300174,1.409397,1.086084,0.782612,1.443087,1.459372,0.837659
8,1.646642,0.550639,2.205562,2.364052,1.930379,1.212686,1.350934,1.819878,1.162012,3.997423,...,1.118806,1.478006,1.160884,1.300424,1.409172,1.075635,0.779935,1.416523,1.459058,0.829848


In [30]:
predict.columns = info_select.columns
pd.DataFrame(predict).to_csv('../output/predict.csv')

In [32]:
test_select = info['2018-12-11':'2018-12-20']
test_select.to_csv('../output/test_select.csv')

In [40]:
# index_date = test_select.index
index_date

DatetimeIndex(['2018-12-11', '2018-12-12', '2018-12-13', '2018-12-14',
               '2018-12-17', '2018-12-18', '2018-12-19', '2018-12-20'],
              dtype='datetime64[ns]', name='date', freq=None)